In [1]:
driver_filepath = r"C:\Users\Jim\Downloads\chromedriver_win32/chromedriver.exe"
save_to_file = True
folder_to_save_to = r"C:\Users\Jim\Documents\fantasy prediction/"

#Import all neccessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from selenium import webdriver
import urllib.request
import json
import urllib
import re
import requests
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from itertools import product
from bs4 import BeautifulSoup

Using TensorFlow backend.


In [2]:
#define a fuzzy merge function
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=1):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

In [3]:
#Get list of fpl player names and ids: players_df
fpl_url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
response=requests.get(fpl_url)
fpl_response = json.loads(response.content)

players = fpl_response['elements']
players_df_whole = pd.DataFrame.from_dict(players)
players_df = players_df_whole[['team', 'web_name', 'id', 'chance_of_playing_next_round','now_cost', 'minutes', 'first_name', 'second_name']]
players_df.dropna()

teams = fpl_response['teams']
teams_df_whole = pd.DataFrame.from_dict(teams)
teams_df = teams_df_whole[['name', 'id']]
players_df = pd.merge(players_df, teams_df, how='left', left_on = ['team'], right_on = ['id'])
players_df['team_name'] = players_df['name']
players_df = players_df[['team', 'web_name', 'id_x', 'chance_of_playing_next_round','now_cost', 'minutes', 'team_name', 'first_name', 'second_name']]
players_df['full_name'] = players_df['first_name'] + players_df['second_name']

In [4]:
#Get this season's odds data: odds_data
driver = webdriver.Chrome(driver_filepath)
driver.implicitly_wait(30)
driver.get(r"https://www.oddsportal.com/soccer/england/premier-league/results/")
c_odds_df=pd.read_html(driver.find_element_by_id("tournamentTable").get_attribute("outerHTML"))
c_odds_df=c_odds_df[0]
if len(c_odds_df) != 1:
    c_odds_df.columns = c_odds_df.columns.droplevel(0)
    c_odds_df.columns = ['1', '2', '3', '4', '5', '6', '7']       

    for i in range(2, 9):
        url = r"https://www.oddsportal.com/soccer/england/premier-league/results/#/page/" + str(i) + r"/"
        driver = webdriver.Chrome(driver_filepath)
        driver.implicitly_wait(30)
        driver.get(url)
        tempdf=pd.read_html(driver.find_element_by_id("tournamentTable").get_attribute("outerHTML"))
        tempdf=tempdf[0]
        tempdf.columns = tempdf.columns.droplevel(0)
        tempdf.columns = ['1', '2', '3', '4', '5', '6', '7']
        c_odds_df = c_odds_df.append(tempdf, ignore_index = True)
else:
    print("no completed match data found")
driver = webdriver.Chrome(driver_filepath)
driver.implicitly_wait(30)
driver.get(r"https://www.oddsportal.com/soccer/england/premier-league/")
tempdf=pd.read_html(driver.find_element_by_id("tournamentTable").get_attribute("outerHTML"))
tempdf=tempdf[0]
if len(tempdf) != 1:
    tempdf.columns = tempdf.columns.droplevel(0)
    tempdf.columns = ['1', '2', '3', '4', '5', '6', '7']
    c_odds_df = c_odds_df.append(tempdf, ignore_index = True)
else: 
    print("no fixture data found")
c_odds_df = c_odds_df[c_odds_df["6"].str.contains("/", na= False)]
odds_data = pd.DataFrame(c_odds_df["2"].str.split(' - ',1).tolist(), columns = ['home_team','away_team'])
split_df_1 = pd.DataFrame(c_odds_df["4"].str.split('/',1).tolist(), columns = ['last','first'])
split_df_1["first"] = pd.to_numeric(split_df_1["first"])
split_df_1["last"] = pd.to_numeric(split_df_1["last"])
odds_data["home_odds"] = split_df_1["first"]/split_df_1["last"]
split_df_2 = pd.DataFrame(c_odds_df["6"].str.split('/',1).tolist(), columns = ['last','first'])
split_df_2["first"] = pd.to_numeric(split_df_2["first"])
split_df_2["last"] = pd.to_numeric(split_df_2["last"])
odds_data["away_odds"] = split_df_2["first"]/split_df_2["last"]
odds_data = odds_data.replace({'Tottenham':'Spurs'}, regex=True)
if save_to_file == True:
    odds_data.to_csv((folder_to_save_to + "odds_data.csv"))

no completed match data found


In [5]:
#get odds data for past seasons, season format "yyyy-yyyy"
past_seasons = ["2019-2020", "2018-2019", "2017-2018", "2016-2017"]

for season in past_seasons:
    driver = webdriver.Chrome(driver_filepath)
    driver.implicitly_wait(30)
    driver.get(r"https://www.oddsportal.com/soccer/england/premier-league-" + season +"/results/")
    seasons=pd.read_html(driver.find_element_by_id("tournamentTable").get_attribute("outerHTML"))
    seasons=seasons[0]
    seasons.columns = seasons.columns.droplevel(0)
    seasons.columns = ['1', '2', '3', '4', '5', '6', '7']       

    for i in range(2, 9):
        url = r"https://www.oddsportal.com/soccer/england/premier-league-" + season + "/results/#/page/" + str(i) + r"/"
        driver = webdriver.Chrome(driver_filepath)
        driver.implicitly_wait(30)
        driver.get(url)
        tempdf=pd.read_html(driver.find_element_by_id("tournamentTable").get_attribute("outerHTML"))
        tempdf=tempdf[0]
        tempdf.columns = tempdf.columns.droplevel(0)
        tempdf.columns = ['1', '2', '3', '4', '5', '6', '7']
        seasons = seasons.append(tempdf, ignore_index = True)

    
        seasons = seasons[seasons["6"].str.contains("/", na= False)]
        s_odds_data = pd.DataFrame(seasons["2"].str.split(' - ',1).tolist(), columns = ['home_team','away_team'])
        split_df_1 = pd.DataFrame(seasons["4"].str.split('/',1).tolist(), columns = ['last','first'])
        split_df_1["first"] = pd.to_numeric(split_df_1["first"])
        split_df_1["last"] = pd.to_numeric(split_df_1["last"])
        s_odds_data["home_odds"] = split_df_1["first"]/split_df_1["last"]
        split_df_2 = pd.DataFrame(seasons["6"].str.split('/',1).tolist(), columns = ['last','first'])
        split_df_2["first"] = pd.to_numeric(split_df_2["first"])
        split_df_2["last"] = pd.to_numeric(split_df_2["last"])
        s_odds_data["away_odds"] = split_df_2["first"]/split_df_2["last"]
        s_odds_data = s_odds_data.replace({'Tottenham':'Spurs'}, regex=True)
    if save_to_file == True:
        filepath = (folder_to_save_to + "odds_data_" + season +".csv"
        s_odds_data.to_csv(filepath)
    else:
        odds_data.append(s_odds_data, ignore_index = True)

SyntaxError: invalid syntax (<ipython-input-5-6b9c0d9c8ad7>, line 38)

In [ ]:
#get links for historic fpl data from github, past season format "yyyy-yy"
github_past_seasons = ['2019-20','2016-17', '2017-18', '2018-19']
players = []
links = []
season_ = []
for season in github_past_seasons:
    github_url = "https://github.com/vaastav/Fantasy-Premier-League/tree/master/data/" +str(season) + "/players"
    github_data = requests.get(github_url)
    soup = BeautifulSoup(github_data.text)
    soup = soup.findAll("a", {"class" : "js-navigation-open link-gray-dark"})
    for link in soup:
        players.append(link.get("title"))
        links.append(link.get("href"))
        season_.append(season)

links_dataframe = pd.DataFrame(players, columns=['name'])
links_dataframe['link'] = pd.DataFrame(links, columns = ['link'])['link']
links_dataframe['season'] = pd.DataFrame(season_, columns=['season'])['season']



In [ ]:
#need to split into all past seasons here
links_dataframe_201617 = links_dataframe[links_dataframe['season'] == "2016-17"]
links_dataframe_201718 = links_dataframe[links_dataframe['season'] == "2017-18"]
links_dataframe_201819 = links_dataframe[links_dataframe['season'] == "2018-19"]
links_dataframe_201920 = links_dataframe[links_dataframe['season'] == "2019-20"]

In [ ]:
#The following operations are done individually because of my PC's RAM constraints
players_merge_201819 = fuzzy_merge(players_df, links_dataframe_201819, 'full_name', 'name', threshold = 55, limit=1)

In [ ]:
players_merge_201819 = players_merge_201819[players_merge_201819['matches'] != ""]
players_merge_201819['matches'].astype(str)
links_dataframe_201819['name'].astype(str)
players_links_201819 = players_merge_201819.merge(links_dataframe_201819, how = 'left', left_on = 'matches', right_on = 'name')
players_links_201819['fuzz_ratio'] = players_links_201819.apply(lambda x: fuzz.partial_ratio(x['full_name'], x['matches']), axis=1)
players_links_201819 = players_links_201819[players_links_201819['fuzz_ratio'] > 80]
if save_to_file == True:
    players_links_201819.to_csv((folder_to_save_to + "players_links_2018-19.csv")

In [ ]:
players_merge_201718 = fuzzy_merge(players_df, links_dataframe_201718, 'full_name', 'name', threshold = 55, limit=1)

In [ ]:
players_merge_201718 = players_merge_201718[players_merge_201718['matches'] != ""]
players_merge_201718['matches'].astype(str)
links_dataframe_201718['name'].astype(str)
players_links_201718 = players_merge_201718.merge(links_dataframe_201718, how = 'left', left_on = 'matches', right_on = 'name')
players_links_201718['fuzz_ratio'] = players_links_201718.apply(lambda x: fuzz.partial_ratio(x['full_name'], x['matches']), axis=1)
players_links_201718 = players_links_201718[players_links_201718['fuzz_ratio'] > 80]
if save_to_file == True:
    players_links_201718.to_csv((folder_to_save_to + "players_links_2017-18.csv"))

In [ ]:
players_merge_201617 = fuzzy_merge(players_df, links_dataframe_201617, 'full_name', 'name', threshold = 55, limit=1)

In [ ]:
players_merge_201617 = players_merge_201617[players_merge_201617['matches'] != ""]
players_merge_201617['matches'].astype(str)
links_dataframe_201617['name'].astype(str)
players_links_201617 = players_merge_201617.merge(links_dataframe_201617, how = 'left', left_on = 'matches', right_on = 'name')
players_links_201617['fuzz_ratio'] = players_links_201617.apply(lambda x: fuzz.partial_ratio(x['full_name'], x['matches']), axis=1)
players_links_201617 = players_links_201617[players_links_201617['fuzz_ratio'] > 80]
if save_to_file == True:
    players_links_201617.to_csv((folder_to_save_to + "players_links_2016-17.csv")

In [ ]:
players_merge_201920 = fuzzy_merge(players_df, links_dataframe_201920, 'full_name', 'name', threshold = 55, limit=1)

In [ ]:
players_merge_201920 = players_merge_201920[players_merge_201920['matches'] != ""]
players_merge_201920['matches'].astype(str)
links_dataframe_201920['name'].astype(str)
players_links_201920 = players_merge_201920.merge(links_dataframe_201920, how = 'left', left_on = 'matches', right_on = 'name')
players_links_201920['fuzz_ratio'] = players_links_201920.apply(lambda x: fuzz.partial_ratio(x['full_name'], x['matches']), axis=1)
players_links_201920 = players_links_201920[players_links_201920['fuzz_ratio'] > 80]
if save_to_file == True:
    players_links_201920.to_csv((folder_to_save_to + "players_links_2019-20.csv")

In [ ]:
#the odds website loads differently on different machines - requires tailoring
#this formatting was neccessary on my laptop
    #replaces code in cell 4
    c_odds_df = c_odds_df[c_odds_df["2"].str.contains("-", na= False)]
    odds_data = pd.DataFrame(c_odds_df["2"].str.split(' - ',1).tolist(), columns = ['home_team','away_team'])
    c_odds_df = c_odds_df.reset_index()
    odds_data['home_odds'] = 1/c_odds_df["4"]
    odds_data['away_odds'] = 1/c_odds_df["6"]
    
    #replaces code in cell 5
    seasons = seasons[seasons["2"].str.contains("-", na= False)]
    odds_data = pd.DataFrame(seasons["2"].str.split(' - ',1).tolist(), columns = ['home_team','away_team'])
    seasons = seasons.reset_index()
    odds_data['home_odds'] = 1/seasons["4"]
    odds_data['away_odds'] = 1/seasons["6"]
    odds_data = odds_data.replace({'Tottenham':'Spurs'}, regex=True)